In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import time
import pickle 
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

INPUT_SIZE = 4
OUTPUT_SIZE = 2
DROPOUT = 0
HIDDEN_SIZE = 10
BATCH_SIZE = 1000
EPOCH = 1_000

In [ ]:
class GRU(nn.Module):
    """ 
    """
    def __init__(self,input_size, hidden_size, output_size):
        super(GRU, self).__init__()
        self.input_size = input_size
        self.hidden = nn.GRU(  
                            input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=1,
                            batch_first=True,
                            dropout=DROPOUT
        )
    
        self.out = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        output, hn = self.hidden(x)
        output = self.out(output)
        
        # action prediction 
        output = F.softmax(output,dim=-1) 
        return output, hn


class behavior_dataset(Dataset):
    """ 
    """
    def __init__(self,dataframe):
        
        # action one hot transformation 
        action = np.array(dataframe['action'])
        if np.all(action == action[0]):
            action = np.append(action,(1-action[0]))
            action = torch.tensor((action).reshape(len(dataframe) + 1),dtype=int)
            action_onehot = nn.functional.one_hot(action, len(action.unique()))
            # delete last one
            action_onehot = action_onehot[:-1]
        else:
            action = torch.tensor((action).reshape(len(dataframe)),dtype=int)
            action_onehot = nn.functional.one_hot(action, len(action.unique()))
        
        # reward
        reward = torch.tensor((np.array(dataframe['reward'])).reshape(len(dataframe)),dtype=int)
        
        # concatinating reward and action
        reward_action = torch.cat([reward[ :, np.newaxis], action_onehot],1)
        
        # block
        block = torch.tensor((np.array(dataframe['start'])).reshape(len(dataframe)),dtype=int)
        
        # concatinating block with and reward_action
        reward_action_block = torch.cat([block[ :, np.newaxis], reward_action],1)
        
        # adding dummy zeros to the beginning and ignoring the last one
        reward_action_block_shift = nn.functional.pad(reward_action_block,[0,0,1,0])[:-1]
        
        # network input 
        x = reward_action_block_shift
        
        # network output 
        y = action_onehot
  
        self.x = x.type(dtype=torch.float32)
        self.y = y.type(dtype=torch.float32)
        self.len = len(dataframe)

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
  
    def __len__(self):
        return self.len    
    

In [ ]:
def train_model(net, train_loader, test_loader, epochs):
    
    train_loss_array, test_loss_array = [], []
    p_r2_array, norm_ll_array = [], []
    net.to(device)  # move net to GPU
    
    # Use Adam optimizer
    optimizer = optim.Adam(net.parameters(), lr=0.001) 
    criterion = nn.BCELoss()
    
    # Loop over epochs 
    for i in range(epochs):
        
        running_loss = 0
        # Loop over training batches
        for j, (X, y_true) in enumerate(train_loader):
    
            X, y_true = X.to(device), y_true.to(device) # move to GPU
            X = X.reshape(1,X.shape[0],INPUT_SIZE) # reshape to  1 x trials x input_size
            
            optimizer.zero_grad()  # zero the gradient buffers
            y_hat_action, hn = net(X) # forward pass
            y_hat_action = (y_hat_action.view(-1, OUTPUT_SIZE)) # Reshape to (SeqLen x Batch, OutputSize)
            loss = criterion(y_hat_action, y_true)  # compute loss
            loss.backward() # backprop the loss
            optimizer.step() # update the weights 
            running_loss += loss.item()
            
        train_loss_array.append(running_loss/len(train_loader))
        test_running_loss, test_p_r2, norm_ll = eval_net(net,test_loader)
        test_loss_array.append(test_running_loss)
        p_r2_array.append(test_p_r2)
        norm_ll_array.append(norm_ll)        
        net.train()
            
    return net, train_loss_array, test_loss_array, p_r2_array, norm_ll_array

def eval_net(net, test_loader):
    
    criterion = nn.BCELoss()
    
    with torch.no_grad():
        net.eval()
        running_loss = 0
        for j, (X, y_true) in enumerate(test_loader):
            
            X, y_true = X.to(device), y_true.to(device) # move to GPU
            X = X.reshape(1,X.shape[0],INPUT_SIZE) # reshape to  1 x trials x input_size
            out, hn = net(X)
            out = out.view(-1, OUTPUT_SIZE)
            loss = criterion(out, y_true)
            running_loss += loss.item()
            
            # pseudo-r2 1-(L/R) L=likelihood model R = likelihood chance model
            y_true = torch.argmax(y_true,1)
            ll = out.gather(1, y_true.view(-1,1))
            ll = torch.sum(torch.log(ll))
            p_r2 = 1 - ( ll / ( len(y_true)*np.log(0.5) ) ) 
            norm_ll = torch.exp(ll/len(y_true))

    return ( running_loss/len(test_loader) ) , p_r2.item() , norm_ll.item()

In [ ]:
df = pd.read_csv('../data/for_plos.csv')
df = df.rename(columns={'key':'action'})

df.loc[df["action"] == "R1", "action"] = 0
df.loc[df["action"] == "R2", "action"] = 1

df['action'] = df['action'].astype(int)
df['reward'] = df['reward'].astype(int)
df['block'] = df['block'] - 1
df['start'] = (df.block.shift() != df.block).astype(int)

# create unique list of names
UniqueNames = df.ID.unique()
dic = {}
for i in range(101):
    dic[UniqueNames[i]] = i
for i in range(101):
    df.loc[df.ID == UniqueNames[i], 'ID'] = dic[UniqueNames[i]]
df = df.rename(columns={'ID':'subject'}).copy()

label = []
for i in range(101):
    if 'B' in df[df.subject==i].diag.values[0]:
        label.append(0)
    elif 'D' in df[df.subject==i].diag.values[0]:
        label.append(1)
    else:
        label.append(2)    
z = []
b = []
d = []
h = []
idx = [] 

for i in range(101):
    if label[i] == 0:
        b.append(df[df.subject==i]) 
    elif label[i] == 1:
        d.append(df[df.subject==i])
    else:
        h.append(df[df.subject==i])

        
for i in range(len(b)):
    z.append(b[i])
    idx.append(np.repeat(i,len(b[i])))
    
last_i = i+1
    
for i in range(len(d)):
    z.append(d[i])
    idx.append(np.repeat(last_i+i,len(d[i])))

last_i = last_i+i+1

for i in range(len(h)):
    z.append(h[i])
    idx.append(np.repeat(last_i+i,len(h[i])))

df = pd.concat(z).reset_index().drop(columns='index')
df['subject'] = np.concatenate(idx)

bipolar_data = df[df.subject<33]
depression_data = df[(df.subject>=33) & (df.subject<67)]
healthy_data = df[(df.subject>=67)]

train_res_gru, test_res_gru, test_p_r2_gru, normlized_ll = [],[],[],[]

In [ ]:
# Bipolar
for i in tqdm(range(33)):
    
    train = bipolar_data[(bipolar_data['subject']!=i)].reset_index() # train on all subj!=i
    test = bipolar_data[(bipolar_data['subject']==i)].reset_index() # leave one out i for test

    train_data = behavior_dataset(train)
    test_data = behavior_dataset(test)

    train_loader = DataLoader(train_data,shuffle=False,batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_data,shuffle=False,batch_size=len(test_data))

    rnn = GRU(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, output_size=OUTPUT_SIZE)
    rnn, train_loss, test_loss, test_p_r2, norm_ll = train_model(rnn, train_loader, test_loader, EPOCH)

    train_res_gru.append(train_loss)
    test_res_gru.append(test_loss)
    test_p_r2_gru.append(test_p_r2)
    normlized_ll.append(norm_ll)
    
# Depression
for i in tqdm(range(34)):
    
    i = i+33
    
    train = depression_data[(depression_data['subject']!=i)].reset_index() # train on all subj!=i
    test = depression_data[(depression_data['subject']==i)].reset_index() # leave one out i for test

    train_data = behavior_dataset(train)
    test_data = behavior_dataset(test)

    train_loader = DataLoader(train_data,shuffle=False,batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_data,shuffle=False,batch_size=len(test_data))

    rnn = GRU(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, output_size=OUTPUT_SIZE)
    rnn, train_loss, test_loss, test_p_r2, norm_ll = train_model(rnn, train_loader, test_loader, EPOCH)

    train_res_gru.append(train_loss)
    test_res_gru.append(test_loss)
    test_p_r2_gru.append(test_p_r2)
    normlized_ll.append(norm_ll)
    
# Healthy 
for i in tqdm(range(34)):
    
    i = i+67
    
    train = healthy_data[(healthy_data['subject']!=i)].reset_index() # train on all subj!=i
    test = healthy_data[(healthy_data['subject']==i)].reset_index() # leave one out i for test

    train_data = behavior_dataset(train)
    test_data = behavior_dataset(test)

    train_loader = DataLoader(train_data,shuffle=False,batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_data,shuffle=False,batch_size=len(test_data))

    rnn = GRU(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, output_size=OUTPUT_SIZE)
    rnn, train_loss, test_loss, test_p_r2, norm_ll = train_model(rnn, train_loader, test_loader, EPOCH)

    train_res_gru.append(train_loss)
    test_res_gru.append(test_loss)
    test_p_r2_gru.append(test_p_r2)
    normlized_ll.append(norm_ll)


In [ ]:
plt.plot(np.array(train_res_gru).reshape(101,EPOCH).mean(axis=0),label='tr_gru')
plt.plot(np.array(test_res_gru).reshape(101,EPOCH).mean(axis=0),label='te_gru')

all_bce_gru = np.zeros(101)
all_pr_2_gru = np.zeros(101)
all_norm_ll_gru = np.zeros(101)

for i in range(33):
    tar = np.array(test_res_gru)[:33][i]
    mask = np.array([False if j==i else True for j in range(33)])
    other = np.array(test_res_gru)[:33][mask]
    ind = np.argmin(other.mean(axis=0))
    all_bce_gru[i] = tar[ind]
    all_pr_2_gru[i] = np.array(test_p_r2_gru).reshape(101,EPOCH)[i][ind]
    all_norm_ll_gru[i] = np.array(normlized_ll).reshape(101,EPOCH)[i][ind]
    
for i in range(34):
    tar = np.array(test_res_gru)[33:67][i]
    mask = np.array([False if j==i else True for j in range(34)])
    other = np.array(test_res_gru)[33:67][mask]
    ind = np.argmin(other.mean(axis=0))
    all_bce_gru[33+i] = tar[ind]
    all_pr_2_gru[33+i] = np.array(test_p_r2_gru).reshape(101,EPOCH)[33+i][ind]
    all_norm_ll_gru[33+i] = np.array(normlized_ll).reshape(101,EPOCH)[33+i][ind]
    
for i in range(34):
    tar = np.array(test_res_gru)[67:][i]
    mask = np.array([False if j==i else True for j in range(34)])
    other = np.array(test_res_gru)[67:][mask]
    ind = np.argmin(other.mean(axis=0))
    all_bce_gru[67+i] = tar[ind]
    all_pr_2_gru[67+i] = np.array(test_p_r2_gru).reshape(101,EPOCH)[67+i][ind]
    all_norm_ll_gru[67+i] = np.array(normlized_ll).reshape(101,EPOCH)[67+i][ind]


plt.axhline(y=all_bce_gru.mean(), ls='--',color=sns.color_palette('tab10')[1],label='gru')
plt.axhline(y=-np.log(.5),ls='--',color='r',label='random_baseline')
plt.axhline(y=0.511,ls='--',color='k',label='qp')
plt.legend()

In [ ]:
diag = np.concatenate([np.repeat(df.diag.unique()[0],33),
        np.repeat(df.diag.unique()[1],34),
        np.repeat(df.diag.unique()[2],34)])

pd.DataFrame({
            'subject':np.arange(101),
            'diag':diag,
            'bce':all_bce_gru,
            'psr2':all_pr_2_gru,
            'norm_ll':all_norm_ll_gru}).to_csv('../results/dezfouli_drnn.csv',index=False)